In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from data_utils import Dictionary, Corpus

In [2]:
#超参数
embed_size = 128
hidden_size = 1024
num_layes = 1
num_epochs = 5
num_samples = 1000
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [3]:
#导入数据
train_path = './data/train.txt'
sample_path = './data/sample.txt'
corpus = Corpus()
ids = corpus.get_data(train_path, batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1)

In [16]:
#创建RNN模型
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layes):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layes, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.init_weights()
        
    def init_weights(self):
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        
    def forward(self, x, h):
        x = self.embed(x)
        
        out, h = self.lstm(x, h)
        
        out = out.contiguous().view(out.size(0)*out.size(1), out.size(2))
        
        out = self.linear(out)
        return out,h
    

In [17]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layes)
model.cuda()

RNNLM (
  (embed): Embedding(10000, 128)
  (lstm): LSTM(128, 1024, batch_first=True)
  (linear): Linear (1024 -> 10000)
)

In [18]:
#定义损失函数、优化函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
#截断反向传播
def detach(states):
    return [state.detach() for state in states] 

In [20]:
#训练模型
for epoch in range(num_epochs):
    states = (Variable(torch.zeros(num_layes, batch_size, hidden_size)).cuda(), Variable(torch.zeros(num_layes, batch_size, hidden_size)).cuda())
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        inputs = Variable(ids[:, i:i+seq_length]).cuda()
        targets = Variable(ids[:, (i+1):(i+1)+seq_length].contiguous()).cuda()
        
        model.zero_grad()
        states = detach(states)
        outputs, states = model(inputs, states) 
        loss = criterion(outputs, targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [%d/%d], Step[%d/%d], Loss: %.3f, Perplexity: %5.2f' %
                   (epoch+1, num_epochs, step, num_batches, loss.data[0], np.exp(loss.data[0])))

Epoch [1/5], Step[0/46479], Loss: 9.213, Perplexity: 10022.47
Epoch [1/5], Step[100/46479], Loss: 6.043, Perplexity: 421.19
Epoch [1/5], Step[200/46479], Loss: 5.947, Perplexity: 382.62
Epoch [1/5], Step[300/46479], Loss: 5.794, Perplexity: 328.40
Epoch [1/5], Step[400/46479], Loss: 5.685, Perplexity: 294.39
Epoch [1/5], Step[500/46479], Loss: 5.141, Perplexity: 170.84
Epoch [1/5], Step[600/46479], Loss: 5.242, Perplexity: 189.02
Epoch [1/5], Step[700/46479], Loss: 5.379, Perplexity: 216.89
Epoch [1/5], Step[800/46479], Loss: 5.228, Perplexity: 186.37
Epoch [1/5], Step[900/46479], Loss: 5.094, Perplexity: 162.98
Epoch [1/5], Step[1000/46479], Loss: 5.163, Perplexity: 174.75
Epoch [1/5], Step[1100/46479], Loss: 5.335, Perplexity: 207.57
Epoch [1/5], Step[1200/46479], Loss: 5.151, Perplexity: 172.63
Epoch [1/5], Step[1300/46479], Loss: 5.119, Perplexity: 167.20
Epoch [1/5], Step[1400/46479], Loss: 4.806, Perplexity: 122.20
Epoch [1/5], Step[1500/46479], Loss: 5.178, Perplexity: 177.41
Ep

In [22]:
#采样
with open(sample_path, 'w') as f:
    state = (Variable(torch.zeros(num_layes, 1, hidden_size)).cuda(),Variable(torch.zeros(num_layes, 1, hidden_size)).cuda())
    
    prob = torch.ones(vocab_size)
    input = Variable(torch.multinomial(prob, num_samples=1).unsqueeze(1),
                     volatile=True).cuda()

    for i in range(num_samples):
        output, state = model(input, state)
        
        prob = output.squeeze().data.exp().cpu()
        word_id = torch.multinomial(prob, 1)[0]
        
        input.data.fill_(word_id)
        
        # 写入文件
        word = corpus.dictionary.idx2word[word_id]
        word = '\n' if word == '<eos>' else word + ' '
        f.write(word)

        if (i+1) % 100 == 0:
            print('Sampled [%d/%d] words and save to %s'%(i+1, num_samples, sample_path))

Sampled [100/1000] words and save to ./data/sample.txt
Sampled [200/1000] words and save to ./data/sample.txt
Sampled [300/1000] words and save to ./data/sample.txt
Sampled [400/1000] words and save to ./data/sample.txt
Sampled [500/1000] words and save to ./data/sample.txt
Sampled [600/1000] words and save to ./data/sample.txt
Sampled [700/1000] words and save to ./data/sample.txt
Sampled [800/1000] words and save to ./data/sample.txt
Sampled [900/1000] words and save to ./data/sample.txt
Sampled [1000/1000] words and save to ./data/sample.txt


In [23]:
#保存模型
torch.save(model.state_dict(), 'model.pkl')